In [1]:
import nltk
import string
from HMM import unsupervised_HMM

In [2]:
def load_poems(filename):
    
    lines = [] # 2d dictionary, each array is a split + cleaned line
    words = {} # dictionary of a word, and its frequency
    
    file = open(filename, 'r')
    
    for line in file:
        line = line.strip()
        if  len(line) < 10:
            # Too short to be a valid line
            continue
        line = "".join(l for l in line if l not in string.punctuation)
        line = line.lower()
        line = line.split()
        
        lines.append(line)

        for word in line:
            try:
                # add to frequency if the word is already in the dic
                words[word] += 1
            except KeyError:
                # if not, add the word to the dic
                words[word] = 1
    return lines, words

In [3]:
file = "data/shakespeare.txt"
lines, words = load_poems(file)

In [4]:
# get syllable info from syllable_dictionary.txt
def get_syllable():
    # do stuff
    return

In [5]:
def unsupervised_learning(lines, n_states, n_iters):
    '''
    n_iters: Number of iterations we should go through.
    n_states: Number of hidden states our HMM should have.
    '''
    # Train the HMM.
    obs, obs_map =  parse_observations(lines)
    flat_lines = [[item] for sublist in lines for item in sublist]
    HMM = unsupervised_HMM(obs, n_states, n_iters)
    numLines = 14
    for i in range(14): # each poem is 14 lines long
        emission = HMM.generate_emission(10, HMM.indexes) # each line is 10 words long
        return emission

In [9]:
    numLines = 14
    for i in range(14): # each poem is 14 lines long
        emission = HMM.generate_emission(10) # each line is 10 words long
        print(emission)

NameError: name 'HMM' is not defined

In [7]:
unsupervised_learning(lines, 4, 10)

merp 0
merp 1
merp 2
merp 3
merp 4
merp 5
merp 6
merp 7
merp 8
merp 9


AttributeError: 'HiddenMarkovModel' object has no attribute 'indexes'

In [6]:

import tensorflow as tf
import keras
from keras.layers import RNN
from keras.layers import LSTM
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten, Dropout
import HMM_helper 
import re
def parse_observations(lines):

    obs_counter = 0
    obs = []
    obs_map = {}

    for line in lines:
        obs_elem = []

        for word in line:
            word = re.sub(r'[^\w]', '', word).lower()
            if word not in obs_map:
                # Add unique words to the observations map.
                obs_map[word] = obs_counter
                obs_counter += 1

            # Add the encoded word.
            obs_elem.append(obs_map[word])

        # Add the encoded sequence.
        obs.append(obs_elem)

    return obs, obs_map

file = "data/shakespeare.txt"
lines, words = load_poems(file)
obs, obs_map = parse_observations(
lines)


maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(obs) - maxlen, step):
    sentences.append(obs[i: i + maxlen])
    next_chars.append(obs[i + maxlen])
print('nb sequences:', len(sentences))
print(sentences[0])
"""
model = Sequential()
#model.add(Embedding(vocabulary, hidden_size, input_length=num_steps))
model.add(LSTM(hidden_size, input_shape=(maxlen, len(chars))))

model.add(TimeDistributed(Dense(vocabulary)))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])


model.fit_generator(train_data_generator.generate(), len(train_data)//(batch_size*num_steps), num_epochs,
                        validation_data=valid_data_generator.generate(),
                        validation_steps=len(valid_data)//(batch_size*num_steps), callbacks=[checkpointer])

"""

Using TensorFlow backend.


nb sequences: 705
[[0, 1, 2, 3, 4, 5], [6, 7, 8, 9, 10, 11, 12], [13, 14, 15, 16, 17, 18, 19, 20], [21, 22, 23, 10, 24, 21, 25], [13, 26, 27, 28, 29, 30, 31, 32], [33, 34, 35, 36, 37, 38, 39], [40, 41, 42, 43, 44, 45], [34, 46, 34, 47, 28, 34, 48, 46, 49, 50], [26, 6, 51, 52, 15, 53, 54, 55], [56, 57, 58, 28, 15, 59, 60], [61, 29, 30, 62, 63, 34, 64], [56, 22, 65, 66, 67, 68, 69], [70, 15, 71, 72, 73, 74, 75, 76], [28, 77, 15, 53, 78, 18, 15, 79, 56, 80], [81, 82, 83, 84, 85, 34, 86], [56, 87, 88, 89, 68, 34, 8, 90], [34, 91, 92, 93, 94, 95, 96, 52], [97, 76, 41, 98, 99, 100, 101, 102, 103], [104, 105, 106, 43, 107, 34, 108, 45], [43, 107, 15, 109, 100, 34, 110, 111], [28, 112, 61, 29, 30, 88, 113, 32], [114, 115, 116, 117, 56, 118, 119], [120, 121, 122, 119, 123, 34, 8, 124], [125, 26, 126, 127, 74, 128, 129, 100, 130], [84, 131, 132, 133, 56, 134, 132, 135, 136], [137, 21, 108, 18, 138, 29], [74, 114, 28, 76, 139, 140, 81, 26, 51, 135], [56, 141, 34, 142, 143, 81, 26, 144, 145, 146],

"\nmodel = Sequential()\n#model.add(Embedding(vocabulary, hidden_size, input_length=num_steps))\nmodel.add(LSTM(hidden_size, input_shape=(maxlen, len(chars))))\n\nmodel.add(TimeDistributed(Dense(vocabulary)))\nmodel.add(Activation('softmax'))\nmodel.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])\n\n\nmodel.fit_generator(train_data_generator.generate(), len(train_data)//(batch_size*num_steps), num_epochs,\n                        validation_data=valid_data_generator.generate(),\n                        validation_steps=len(valid_data)//(batch_size*num_steps), callbacks=[checkpointer])\n\n"

In [9]:
lines

[['from', 'fairest', 'creatures', 'we', 'desire', 'increase'],
 ['that', 'thereby', 'beautys', 'rose', 'might', 'never', 'die'],
 ['but', 'as', 'the', 'riper', 'should', 'by', 'time', 'decease'],
 ['his', 'tender', 'heir', 'might', 'bear', 'his', 'memory'],
 ['but', 'thou', 'contracted', 'to', 'thine', 'own', 'bright', 'eyes'],
 ['feedst', 'thy', 'lights', 'flame', 'with', 'selfsubstantial', 'fuel'],
 ['making', 'a', 'famine', 'where', 'abundance', 'lies'],
 ['thy', 'self', 'thy', 'foe', 'to', 'thy', 'sweet', 'self', 'too', 'cruel'],
 ['thou', 'that', 'art', 'now', 'the', 'worlds', 'fresh', 'ornament'],
 ['and', 'only', 'herald', 'to', 'the', 'gaudy', 'spring'],
 ['within', 'thine', 'own', 'bud', 'buriest', 'thy', 'content'],
 ['and', 'tender', 'churl', 'makst', 'waste', 'in', 'niggarding'],
 ['pity', 'the', 'world', 'or', 'else', 'this', 'glutton', 'be'],
 ['to', 'eat', 'the', 'worlds', 'due', 'by', 'the', 'grave', 'and', 'thee'],
 ['when', 'forty', 'winters', 'shall', 'besiege', 'thy